In [31]:
import os,sys,progressbar
import numpy as np
import healpy as hp
import pandas as pd
import astropy.io.fits as fits
import time
from IPython.display import clear_output

sys.path.insert(1, '/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/scripts')
import gmca4im_lib2 as g4i
import Extension4BINGO as cs

In [11]:
#######################################################
################   GENERAL INFORMATIONS   #############
#######################################################
method           = "ICA"
wtransform       = "pywavelets"
maps_wout_mean   = True
apply_mask       = False
add_noise        = False
#######################################################
################   WAVELETS PARAMETERS   ##############
#######################################################
#####
# The main aim is to have:
# Starlet, Axisymmetric, spin-Directional, Wavelets Standard from PyWavelets, Curvelets, Counturlets, Shearlets, Ridgelets and so on
#####
J     = 1  #number of scales
use_c = True  # if you will use wavelet scale in the analysis
##############
# S2LET code
# If you to use wtransforms by S2Let code, please, fill in the variables below:
L        = None #If you write "None", it will use L=3*nside
J_min    = 1
B        = 5     
N        = 3  # Number of directions (This is for Directional only)
spin     = 0  # set to 0 for temperature. if non-zero, plotting routines must be changed! (This is for Directional only)
upsample = 0  # 1 means all scales at full resolution L # 0 means multiresolution wavelet transform (This is for Directional only)
# In the S2LET code, J scales is defined by code and not by J above.

##############
# PyWavelets
Jpwt = 2 #number of scales
pywttype = "db1" 
##############
# Curvelets

##############
# Counturlets

##############
# Shearlets

##############
# Ridgelets

#######################################################
#### COMPONENT SEPARATION #############################
#######################################################
n_s = 3  #number of sources to be estimated
######## FastICA PARAMETERS 
whiten = True  ######## Maintain True
fun = 'logcosh' #exp,logcosh or
max_iter = 100 
tol = 0.0001
######## GMCA PARAMETERS   
mints = 0.05 # min threshold (what is sparse compared to noise?)
nmax  = 100 # number of iterations (usually 100 is safe)
L0    = 0   # switch between L0 norm (1) or L1 norm (0)
#######################################################
AInit     = None
ColFixed  = None
whitening = False
epsi      = 1e-3
verbose   = False
#GMCAExtension
div          = 1 #  J+1  #J/div will should be even number
without_covx = True # if your mixmatrix estimated will use covariance matrix of the observer data with ponderation
#######################################################
################   DEBIAS PARAMETERS   ################
#######################################################
seed_used = 10
#######################################################
################   PATHS PARAMETERS   #################
#######################################################
#path outputs
pathout       = "/home/marins/Documents/Programmation/BINGO/Component Separation/gmca4im/test_new_module/GMCAn3_haarJ2_test" #Put here your path to the output cls
cl_type_save  = "reconstruction" #You should choice between reconstruction or residuals cls values
#######################################################
################   NAME FILES PARAMETERS   ############
#######################################################
# Name of FITS files inside of the pathmaps
name_mask = "Mask_Bin.fits" #put this file in the same directory of the other maps

#Directory names
dir_observed  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/Cubos_Input_L10_L25_21cm_fg_White_Noise"
dir_mask      = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/mask"
dir_prior     = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/Cubos_Prior_WN" 
dir_noise     = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/wn_masked"         #Put here directory name of the noise maps 
dir_pure      = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/Cubos_21cm_Masked" #Put here directory name of the pure maps 
dir_projprior = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/Cubos_Prior_WN"    #Put here directory name of the prior maps
dir_projnoise = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/wn_masked"         #Put here directory name of the noise maps
dir_projpure  = "/home/marins/Documents/Programmation/BINGO/Component Separation/maps/Cubos_21cm_Masked" #Put here directory name of the pure maps 

In [12]:
params_maps = pd.Series({"without_mean":maps_wout_mean, "apply_mask":apply_mask, "add_noise":add_noise, "cl_type_save":cl_type_save})
params_CS   = pd.Series({"method":method,
                         "A_ini":AInit, "ColFixed":ColFixed, "whitening":whitening, "epsi":epsi, "verbose":verbose, "ns":n_s, "mints":mints,"nmax":nmax, "L0":L0, "division":div, "without_covx":without_covx, "seed_used":seed_used,
                         "whiten":whiten, "fun":fun, "max_iter":max_iter, "tol":tol})
params_WT   = pd.Series({"wtransform":np.asarray(wtransform), "use_c":use_c, "J":J, 
                         "L":L, "Jmin":J_min, "B": B, "N":N, "spin":spin, "upsample":upsample,
                         "Jpwt":Jpwt, "pywttype":pywttype.lower()})
params_path = pd.Series({"pathout":pathout, "dir_observed":dir_observed, "dir_mask":dir_mask, "dir_noise":dir_noise, "dir_prior":dir_prior,"dir_pure":dir_pure, "name_mask":name_mask})

### Foregrounds + HI + Noise
$C_{\ell}^{\textrm{FG+HI+N}}$

In [49]:
names = np.asarray(os.listdir(params_path.dir_observed))
nseed = cs.extracting_seed_from_filenames(vectornames=names)


timei = time.time()
params_maps["getdata"] = "observed"
subdirs = cs.checkdir(params_path.pathout, subdirs=["21cm","foregrounds","mixmatrix"])

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_observed"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    X = cs.maps2CSmaps(X, params_WT, params_CS)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)
    del X
    time0   = time.time()-time0
    #print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)
bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:10:20 Time:  0:10:20


### Noise
$C_{\ell}^{\textrm{N}}$

In [43]:
names = np.asarray(os.listdir(params_path.dir_noise))
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "noise"
subdirs = ["noise"]

if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:03:17 Time:  0:03:17


### Prior
$C_{\ell}^{\textrm{prior}}$

In [44]:
names = np.asarray(os.listdir(params_path.dir_prior))
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "prior"
subdirs = ["prior"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

bar = progressbar.ProgressBar(maxval=names.size)    
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:03:09 Time:  0:03:09


### 21cm pure
$C_{\ell}^{\textrm{pure}}$

In [45]:
names = np.asarray(os.listdir(params_path.dir_pure))
#nseed = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "pure"
subdirs = ["pure"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    
bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:03:00 Time:  0:03:00


### 21cm proj pure
$C_{\ell}^{\textrm{proj pure}}$

In [46]:
names = np.asarray(os.listdir(params_path.dir_pure))
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "pure"
subdirs = ["projpure"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    


bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:02:21 Time:  0:02:21


### 21cm  proj noise
$C_{\ell}^{\textrm{proj noise}}$

In [47]:
names = np.asarray(os.listdir(params_path.dir_noise))
nseed = cs.extracting_seed_from_filenames(vectornames=names)
timei   = time.time()
params_maps["getdata"] = "noise"
subdirs = ["projnoise"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    


bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:03:29 Time:  0:03:29


### 21cm  proj prior
$C_{\ell}^{\textrm{proj prior}}$

In [48]:
names   = np.asarray(os.listdir(params_path.dir_prior))
#nseed   = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "prior"
subdirs = ["projprior"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (16 of 16) |########################| Elapsed Time: 0:03:20 Time:  0:03:20
